<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 40px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Chatbot - Génératif - Agnostique
  </div> 
  
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  M2DS 2019
  </div> 
  

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  Jean-baptiste Aujogue
  </div>

In [1]:
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import re
import random
import pickle
import copy
# package a installer d'abord avec anaconda
#import spacy

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# package a installer d'abord avec anaconda
import gensim
from gensim.models import KeyedVectors

#import nltk
#nltk.download()
#from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import unidecode
from unidecode import unidecode

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import time
import math
import re
import json

In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [4]:
import sys
sys.path.append('C:\\Users\\Jb\\Desktop\\NLP\\chatNLP')

# 1 Training data preparation

### 1.1 Simple formatting of training dialogues

In [5]:
# -------------------- enlèvement des stopwords ---------------------------------------
def TrimWordsSentence(sentence, stopwords):
    '''Remove stopwords from a sentence'''
    resultwords = [word for word in sentence if word.lower() not in stopwords]
    return resultwords

def TrimWordsDialogue(dialogue, stopwords):
    '''Remove stopwords from user utterances in a dialogue'''
    for pair in dialogue: 
        pair[0] = TrimWordsSentence(pair[0], stopwords)
        #pair[1] = pair[1].strip()
    return dialogue

def TrimWords(dialogues, stopwords):
    '''Remove stopwords from user utterances in a list of dialogues'''
    return [TrimWordsDialogue(dialogue, stopwords) for dialogue in dialogues ]

    


# --------------------------- Normalisation -------------------------------
def normalizeString(s):
    '''Remove rare symbols from a string'''
    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
 
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", s) 
    s = re.sub("\(", r" ( ", s)
    s = re.sub("\)", r" ) ", s)
    s = re.sub(r"\.", r" . ", s)
    s = re.sub(r",", r" , ", s)
    s = re.sub(r"!", r" ! ", s)
    s = re.sub(r":", r" : ", s)
    s = re.sub(r"-", r" - ", s)
    s = re.sub(r"'", r" ' ", s)
    s = re.sub(r";", r" ; ", s)
    s = re.sub(r' +', r' ', s).strip()
    return s 



#--------------------- import des dialogues --------------------
def importDialogues(fichier, limite = None):
    '''Import a textfile containing dialogues and returns a list, each element 
       corresponding to a dialogue and also being under the form of a list, with 
       each element being a list of two elements : an element giving a user 
       utterance and another element giving the bot response. Both elements are 
       normalized strings.
       Ex. The dialogue :
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
       now becomes :
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
       Lines corresponding to user utterance with no bot response are discarted.
    '''
    def cleanS(s):
        cleans = normalizeString(s).split(' ')
        cleans = [word for word in cleans if word.strip() != '']
        return cleans
    dialogues_import = open(fichier, encoding='utf-8').read().strip().split('\n\n')
    dialogues = []
    for i, d in enumerate(dialogues_import):
        dialogue = []
        lines = d.split('\n')
        for l in lines:
            if len(l.split('\t')) == 2 :
                pair = [cleanS(s) for s in l.split('\t')]
                dialogue.append(pair)
            elif len(l.split('\t')) == 3 :
                pair = [cleanS(s) for s in l.split('\t')[:2]]
                dialogue.append(pair)
        dialogues.append(dialogue)
        if limite is not None and i == limite -1 :
            break

    return dialogues






#------------------ Dictionnaire des mots variables -----------------------------
def motVar(file):
    '''Applies to the Master's program dataset.
       Import the collection of pairs token-content for a set of variable words.
    '''
    lines = open(file, encoding='utf-8').read().strip().split('\n')
    motsVar = {}
    for l in lines :
        cle, valeur = l.split('\t')
        motsVar[cle.lower()] = valeur
    return motsVar




# ----------------------- Création de la liste des dialogues--------------------
def prepareData(opt):
    '''Import dialogue from text file and apply some formatting operations,
       as described in the functions 
               - importDialogues
               - modify
               - TrimWords
               - filterDialogues
    '''
    dialogues = importDialogues(fichier = opt['fichier'], 
                                limite = opt['limite'])
    dialogues = modify(dialogues) if opt['modify'] else dialogues
    dialogues = TrimWords(dialogues, opt['stopwords']) # on enlève les stopwords de chaque question
    print(" %s dialogues ..." % len(dialogues))
    print(dialogues[0])
    if opt['filtre'] :
        #for pair in [pair for pair in pairs if not filterPair(pair)]:
        #    print('%s (%d) -> %s (%d)' % (pair[0],len(pair[0].split()),pair[1],len(pair[1].split())))  
        dialogues = filterDialogues(dialogues, opt['max_length'])
        print('')
        print("... reduced to %s dialogues" % len(dialogues))

    return dialogues

In [6]:
motsVar = motVar('C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\chatbot-M2-DS-Variables.txt')

Turn each dialogue of the corpus into torch variables

In [7]:
def variableFromSentence(lang, sentence, rand = 0): # sentence = [str]
    indexes=[]
    unknowns = 0
    for word in sentence:
        p = random.random()
        if word not in lang.word2index.keys() and 'UNK' in lang.word2index.keys() :
            #pass
            indexes.append(lang.word2index['UNK'])
        elif p >= rand :
            indexes.append(lang.word2index[word])
        elif p < rand :
            e = random.choice([1, 2])
            if e == 1 :  # doesn't put any word
                pass
            elif e == 2 and 'UNK' in lang.word2index.keys() :# hide word with UNK_Token
                indexes.append(lang.word2index['UNK'])
    indexes.append(lang.word2index['EOS'])                                
    result = Variable(torch.LongTensor([[i] for i in indexes]))
    return result

def variableFromDialogue(lang, dialogue, rand = 0): # sentence = [str]
    result = []
    for paire in dialogue :
        el1 = variableFromSentence(lang, paire[0], rand = rand)
        el2 = variableFromSentence(lang, paire[1], rand = 0)
        result.append([el1, el2])
    return result

def variableFromAllDialogues(lang, dialogues, rand = 0): # sentence = [str]
    result = []
    for dialogue in dialogues :
        result.append(variableFromDialogue(lang, dialogue, rand))
    return result

### 1.2 Language class

In [8]:
class Lang:
    def __init__(self, name, init = 3):
        self.name = name
        if init == 0 :
            self.word2index = {}
            self.word2count = {}
            self.index2word = {}
            self.n_words = 0
        elif init == 3 :
            self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
            self.word2count = {"SOS": 0, "EOS": 0, "UNK": 0}
            self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
            self.n_words = 3

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            if word.strip() != '' :
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in nltk.word_tokenize(sentence): # sentence.split()
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions, lvl = 1):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        if lvl == 2 :
                            for sentence in nltk.sent_tokenize(description) :
                                self.addSentence(sentence)
                        else :
                            self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))
            
            
    def addDialogues(self, dialogues, i):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for dialogue in dialogues :
            for pair in dialogue:
                try :
                    self.addSentence(pair[i])
                except IndexError:
                    print("Problem with {}".format(pair))

In [9]:
def generateLanguages(dialogues):
    '''Generate three languages classes out of a list of dialogues :
            - input_lang containing the user's vocabulary
            - output lang containing the bot vocabulary
            - output_sentence_lang containing the bot answers as words of a vocabulary
    '''
    lang = Lang('M2DS')
    
    lang.addDialogues(dialogues, 0)
    lang.addDialogues(dialogues, 1)
    print("Mots comptés :")
    print(lang.name, lang.n_words)
    
    return lang


def ajout(dialogues, lang, i= 1):
    '''addDialogues method of the Lang class with prints.'''
    lang.addDialogues(dialogues, i)
    print(lang.name, lang.n_words)
    return lang 


def ajoutSentences(dialogues, sentences_lang, i = 1) :
    '''Add sentences as words to a given language'''
    for dialogue in dialogues :
        for pair in dialogue :
            try :
                sentences_lang.addWord(pair[i])
            except IndexError:
                print("Problem with {}".format(pair))
                
    return sentences_lang

## 1.3 Master's program dataset

In [11]:
MAX_LENGTH = 100
max_length = MAX_LENGTH
stopwords = []

Master_train =   {'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\Conversations_M2DS_2019_final\\ChatbotDS_Train.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'filtre' : False}

dialogues_Master = prepareData(Master_train)

Master_test  =   {'lang1': 'lang_client',
                   'lang2' : 'lang_agent',
                   'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\Conversations_M2DS\\Conversations_M2DS_2019_final\\ChatbotDS_Test.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'reverse' : False,
                   'filtre' : False}

dialogues_Master_test = prepareData(Master_test)

 100000 dialogues ...
[[['quel', 'est', 'l', "'", 'enseignant', 'de', 'l', "'", 'ue', 'de', 'pgm', '?'], ['l', "'", 'enseignant', 'de', 'modeles', 'graphiques', 'probabilistes', 'est', 'pr', '.', 'profpgm']], [['ca', 'va', 'merci', 'j', "'", 'ai', 'besoin', 'de', 'rien'], ['d', "'", 'accord', ',', 'j', "'", 'espere', 'avoir', 'correctement', 'repondu', 'a', 'vos', 'questions', '.']], [['que', 'faut', 'il', 'lire', 'en', 'statistique', 'inferentielle', '?'], ['vous', 'devriez', 'prendre', 'contact', 'avec', 'l', "'", 'enseignant', 'de', 'l', "'", 'ue', 'et', 'lui', 'demander']], [['je', 'viens', 'de', 'l', "'", 'etranger'], ['d', "'", 'accord', ',', 'comment', 'puis', '-', 'je', 'vous', 'aider', '?']], [['quel', 'est', 'le', 'cout', 'du', 'master', '?'], ['vous', 'devriez', 'poser', 'la', 'question', 'au', 'service', 'de', 'scolarite', '.']], [['c', "'", 'est', 'quand', 'la', 'fin', 'des', 'cours', '?'], ['aux', 'alentours', 'du', 'mois', 'de', 'mars', '.']], [['ca', 'va', 'merci', 'j',

In [12]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

1364


In [13]:
lang_M2DS = generateLanguages(dialogues_Master)

Mots comptés :
M2DS 1291


In [ ]:
# compute ONCE
#with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang_M2DS_2019_final.file', 'wb') as fileObject :
#    pickle.dump(lang_M2DS, fileObject)

# 1062 / 1272 / 1291 mots comptés

In [ ]:
def importLang(name):
    with open('C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang\\'+name+'.file', 'rb') as fil :
        lang = pickle.load(fil)
    return lang

In [ ]:
#lang_M2DS = importLang('lang_M2DS_2019_final') # 1062 / 1272 / 1291

# 2 Word Embedding Pretraining

## 2.1 Models

In [14]:
from chatNLP.models.Word_Embedding import CBOW, SkipGram, WordEmbeddingTrainer

### 2.1.1 Cbow

In [16]:
cbow = CBOW(device,
            lang = lang_M2DS,
            context_size = 5,
            embedding_dim = 100,
            hidden_dim = 100)
cbow = cbow.to(device)

In [ ]:
cbow.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_cbow.pth'))

### 2.1.2 SkipGram

**Data**

In [18]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


def generateNgrams(descriptions, context_size = 5, word_first = True) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        lines = flatten(description)
        line =  flatten(lines)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i] + line[i+1 : i+context_size+1]
            target = [line[i]]
            if word_first :
                data.append((target, context))
            else :
                data.append((context, target))
    return data

**Model**

In [15]:
skipgram = SkipGram(device, 
                    lang = lang_M2DS, 
                    context_size = 5, 
                    embedding_dim = 100, 
                    hidden_dim = 150)
skipgram = skipgram.to(device)

In [16]:
skipgram.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_skipgram_100.pth'))

**Training**

In [19]:
word_embedding_trainer = WordEmbeddingTrainer(device = device)

In [22]:
for alpha in [0.01, 0.005, 0.0025, 0.001] :
    for i in range(20) :
        Ngrams = generateNgrams(dialogues_Master[5000*i:5000*(i+1)], context_size = 5, word_first = True)
        word_embedding_trainer.train(skipgram, Ngrams, n_iters = 5000, learning_rate=alpha)

0m 0s (- 0m 21s) (100 2%) 7.1594 9.98
0m 0s (- 0m 22s) (200 4%) 7.0452 9.82
0m 1s (- 0m 21s) (300 6%) 6.9597 9.68
0m 1s (- 0m 21s) (400 8%) 6.8505 9.45
0m 2s (- 0m 21s) (500 10%) 6.7443 9.51
0m 2s (- 0m 20s) (600 12%) 6.6424 9.61
0m 3s (- 0m 20s) (700 14%) 6.5610 9.62
0m 3s (- 0m 19s) (800 16%) 6.4343 9.56
0m 4s (- 0m 19s) (900 18%) 6.4177 9.61
0m 4s (- 0m 18s) (1000 20%) 6.3094 9.58
0m 5s (- 0m 18s) (1100 22%) 6.2583 9.54
0m 5s (- 0m 17s) (1200 24%) 6.1976 9.43
0m 6s (- 0m 17s) (1300 26%) 6.1901 9.66
0m 6s (- 0m 16s) (1400 28%) 6.1105 9.57
0m 7s (- 0m 16s) (1500 30%) 6.0057 9.48
0m 7s (- 0m 16s) (1600 32%) 5.9809 9.46
0m 8s (- 0m 15s) (1700 34%) 5.8727 9.50
0m 8s (- 0m 15s) (1800 36%) 5.9425 9.57
0m 8s (- 0m 14s) (1900 38%) 5.8525 9.39
0m 9s (- 0m 14s) (2000 40%) 5.8764 9.68
0m 9s (- 0m 13s) (2100 42%) 5.8636 9.53
0m 10s (- 0m 13s) (2200 44%) 5.7840 9.41
0m 10s (- 0m 12s) (2300 46%) 5.7784 9.44
0m 11s (- 0m 12s) (2400 48%) 5.7474 9.43
0m 11s (- 0m 11s) (2500 50%) 5.7906 9.48
0m 12s (-

0m 3s (- 0m 20s) (700 14%) 5.0561 8.75
0m 3s (- 0m 19s) (800 16%) 4.8894 8.61
0m 4s (- 0m 19s) (900 18%) 4.9069 8.50
0m 4s (- 0m 19s) (1000 20%) 4.6914 8.27
0m 5s (- 0m 18s) (1100 22%) 4.9471 8.71
0m 5s (- 0m 18s) (1200 24%) 5.0160 8.70
0m 6s (- 0m 17s) (1300 26%) 4.8829 8.63
0m 6s (- 0m 17s) (1400 28%) 5.0645 8.79
0m 7s (- 0m 16s) (1500 30%) 4.8150 8.40
0m 7s (- 0m 16s) (1600 32%) 4.7054 8.37
0m 8s (- 0m 15s) (1700 34%) 4.7635 8.39
0m 8s (- 0m 15s) (1800 36%) 4.7259 8.62
0m 9s (- 0m 14s) (1900 38%) 4.7044 8.28
0m 9s (- 0m 14s) (2000 40%) 4.7505 8.41
0m 9s (- 0m 13s) (2100 42%) 4.8755 8.72
0m 10s (- 0m 13s) (2200 44%) 5.0200 8.81
0m 10s (- 0m 12s) (2300 46%) 4.7184 8.38
0m 11s (- 0m 12s) (2400 48%) 4.9043 8.70
0m 11s (- 0m 11s) (2500 50%) 4.8362 8.48
0m 12s (- 0m 11s) (2600 52%) 4.8654 8.78
0m 12s (- 0m 10s) (2700 54%) 4.9942 8.97
0m 13s (- 0m 10s) (2800 56%) 4.7940 8.25
0m 13s (- 0m 9s) (2900 57%) 4.7298 8.39
0m 14s (- 0m 9s) (3000 60%) 4.9214 8.71
0m 14s (- 0m 8s) (3100 62%) 4.7805 8

0m 6s (- 0m 17s) (1300 26%) 4.5776 8.19
0m 6s (- 0m 16s) (1400 28%) 4.4845 8.03
0m 7s (- 0m 16s) (1500 30%) 4.3223 7.92
0m 7s (- 0m 16s) (1600 32%) 4.2603 7.81
0m 8s (- 0m 15s) (1700 34%) 4.6256 8.33
0m 8s (- 0m 15s) (1800 36%) 4.6110 8.27
0m 8s (- 0m 14s) (1900 38%) 4.5013 8.02
0m 9s (- 0m 14s) (2000 40%) 4.3780 8.07
0m 9s (- 0m 13s) (2100 42%) 4.4514 7.95
0m 10s (- 0m 13s) (2200 44%) 4.4938 8.07
0m 10s (- 0m 12s) (2300 46%) 4.5416 8.38
0m 11s (- 0m 12s) (2400 48%) 4.5946 8.24
0m 11s (- 0m 11s) (2500 50%) 4.3505 7.75
0m 12s (- 0m 11s) (2600 52%) 4.3954 8.04
0m 12s (- 0m 10s) (2700 54%) 4.5276 8.12
0m 13s (- 0m 10s) (2800 56%) 4.7808 8.69
0m 13s (- 0m 9s) (2900 57%) 4.4101 7.97
0m 14s (- 0m 9s) (3000 60%) 4.3978 7.97
0m 14s (- 0m 8s) (3100 62%) 4.4204 8.21
0m 15s (- 0m 8s) (3200 64%) 4.5013 8.28
0m 15s (- 0m 8s) (3300 66%) 4.5139 8.17
0m 16s (- 0m 7s) (3400 68%) 4.5528 8.21
0m 16s (- 0m 7s) (3500 70%) 4.5080 8.11
0m 16s (- 0m 6s) (3600 72%) 4.5392 8.38
0m 17s (- 0m 6s) (3700 74%) 4.540

0m 8s (- 0m 14s) (1900 38%) 4.3775 8.08
0m 9s (- 0m 13s) (2000 40%) 4.4017 7.92
0m 9s (- 0m 13s) (2100 42%) 4.4008 7.99
0m 10s (- 0m 13s) (2200 44%) 4.3468 8.16
0m 10s (- 0m 12s) (2300 46%) 4.1667 7.78
0m 11s (- 0m 12s) (2400 48%) 4.5333 8.18
0m 11s (- 0m 11s) (2500 50%) 3.9663 7.35
0m 12s (- 0m 11s) (2600 52%) 4.2298 7.93
0m 12s (- 0m 10s) (2700 54%) 4.2475 7.94
0m 13s (- 0m 10s) (2800 56%) 4.2008 7.84
0m 13s (- 0m 9s) (2900 57%) 4.2457 8.04
0m 14s (- 0m 9s) (3000 60%) 4.3257 7.97
0m 14s (- 0m 9s) (3100 62%) 4.2536 7.84
0m 15s (- 0m 8s) (3200 64%) 4.1494 7.45
0m 15s (- 0m 8s) (3300 66%) 4.4451 8.25
0m 16s (- 0m 7s) (3400 68%) 4.4235 8.21
0m 16s (- 0m 7s) (3500 70%) 4.3650 8.07
0m 16s (- 0m 6s) (3600 72%) 4.0337 7.51
0m 17s (- 0m 6s) (3700 74%) 4.2600 7.71
0m 17s (- 0m 5s) (3800 76%) 4.2932 7.87
0m 18s (- 0m 5s) (3900 78%) 4.4843 8.33
0m 19s (- 0m 4s) (4000 80%) 4.3821 8.19
0m 19s (- 0m 4s) (4100 82%) 4.1174 7.67
0m 20s (- 0m 3s) (4200 84%) 4.2081 7.79
0m 20s (- 0m 3s) (4300 86%) 4.380

0m 11s (- 0m 12s) (2400 48%) 3.9475 7.82
0m 11s (- 0m 11s) (2500 50%) 4.1049 7.66
0m 12s (- 0m 11s) (2600 52%) 4.3373 8.14
0m 12s (- 0m 10s) (2700 54%) 3.9633 7.46
0m 13s (- 0m 10s) (2800 56%) 4.0356 7.18
0m 13s (- 0m 9s) (2900 57%) 4.0953 7.79
0m 13s (- 0m 9s) (3000 60%) 4.2054 7.85
0m 14s (- 0m 8s) (3100 62%) 4.1204 7.80
0m 14s (- 0m 8s) (3200 64%) 4.2569 8.13
0m 15s (- 0m 7s) (3300 66%) 4.1097 7.85
0m 15s (- 0m 7s) (3400 68%) 4.1384 7.37
0m 16s (- 0m 6s) (3500 70%) 4.1052 7.78
0m 16s (- 0m 6s) (3600 72%) 4.1633 7.84
0m 17s (- 0m 6s) (3700 74%) 4.1667 7.97
0m 17s (- 0m 5s) (3800 76%) 4.1449 7.90
0m 18s (- 0m 5s) (3900 78%) 3.8902 7.56
0m 18s (- 0m 4s) (4000 80%) 4.0855 7.89
0m 19s (- 0m 4s) (4100 82%) 4.0471 7.82
0m 19s (- 0m 3s) (4200 84%) 4.1001 7.86
0m 20s (- 0m 3s) (4300 86%) 3.9562 7.64
0m 20s (- 0m 2s) (4400 88%) 4.2155 7.92
0m 20s (- 0m 2s) (4500 90%) 4.4333 8.34
0m 21s (- 0m 1s) (4600 92%) 4.5058 8.28
0m 21s (- 0m 1s) (4700 94%) 4.2111 7.77
0m 22s (- 0m 0s) (4800 96%) 4.1564 

...

0m 14s (- 0m 9s) (3000 60%) 3.7596 7.39
0m 14s (- 0m 9s) (3100 62%) 3.6081 7.28
0m 15s (- 0m 8s) (3200 64%) 3.7527 7.60
0m 15s (- 0m 8s) (3300 66%) 3.5942 7.32
0m 16s (- 0m 7s) (3400 68%) 3.8776 7.86
0m 16s (- 0m 7s) (3500 70%) 3.6543 7.57
0m 17s (- 0m 6s) (3600 72%) 3.4553 6.98
0m 17s (- 0m 6s) (3700 74%) 3.9067 7.86
0m 17s (- 0m 5s) (3800 76%) 3.7532 7.54
0m 18s (- 0m 5s) (3900 78%) 3.6957 7.51
0m 18s (- 0m 4s) (4000 80%) 3.6645 7.39
0m 19s (- 0m 4s) (4100 82%) 3.5418 7.36
0m 19s (- 0m 3s) (4200 84%) 3.5642 7.30
0m 20s (- 0m 3s) (4300 86%) 3.5884 7.06
0m 20s (- 0m 2s) (4400 88%) 3.8091 7.42
0m 21s (- 0m 2s) (4500 90%) 3.8404 7.57
0m 21s (- 0m 1s) (4600 92%) 3.6742 7.26
0m 22s (- 0m 1s) (4700 94%) 3.9474 7.81
0m 22s (- 0m 0s) (4800 96%) 3.7020 7.69
0m 23s (- 0m 0s) (4900 98%) 3.4887 7.14
0m 23s (- 0m 0s) (5000 100%) 3.5245 6.90
0m 0s (- 0m 23s) (100 2%) 3.6487 7.16
0m 0s (- 0m 23s) (200 4%) 3.5515 7.33
0m 1s (- 0m 22s) (300 6%) 3.7324 7.52
0m 1s (- 0m 21s) (400 8%) 3.8403 7.78
0m 2s (

0m 18s (- 0m 7s) (3500 70%) 3.6110 7.21
0m 19s (- 0m 7s) (3600 72%) 3.9640 7.81
0m 19s (- 0m 6s) (3700 74%) 3.9986 7.96
0m 20s (- 0m 6s) (3800 76%) 3.5769 7.10
0m 21s (- 0m 5s) (3900 78%) 3.6039 7.28
0m 21s (- 0m 5s) (4000 80%) 3.4820 7.22
0m 22s (- 0m 4s) (4100 82%) 3.7157 7.56
0m 22s (- 0m 4s) (4200 84%) 3.7624 7.52
0m 23s (- 0m 3s) (4300 86%) 3.8353 7.45
0m 23s (- 0m 3s) (4400 88%) 3.6339 7.40
0m 24s (- 0m 2s) (4500 90%) 3.7642 7.59
0m 24s (- 0m 2s) (4600 92%) 3.8708 7.67
0m 25s (- 0m 1s) (4700 94%) 3.5469 7.25
0m 25s (- 0m 1s) (4800 96%) 3.6382 7.10
0m 26s (- 0m 0s) (4900 98%) 3.5187 7.08
0m 26s (- 0m 0s) (5000 100%) 3.6497 7.11
0m 0s (- 0m 29s) (100 2%) 3.6026 7.32
0m 1s (- 0m 29s) (200 4%) 3.5099 7.10
0m 1s (- 0m 28s) (300 6%) 3.5628 7.09
0m 2s (- 0m 27s) (400 8%) 3.5369 7.42
0m 2s (- 0m 26s) (500 10%) 3.8354 7.59
0m 3s (- 0m 27s) (600 12%) 3.8025 7.41
0m 4s (- 0m 27s) (700 14%) 3.4754 7.12
0m 5s (- 0m 27s) (800 16%) 3.6907 7.69
0m 5s (- 0m 26s) (900 18%) 3.8151 7.86
0m 6s (- 0m 

0m 20s (- 0m 5s) (4000 80%) 3.6025 7.28
0m 21s (- 0m 4s) (4100 82%) 3.5368 7.30
0m 21s (- 0m 4s) (4200 84%) 3.7034 7.37
0m 22s (- 0m 3s) (4300 86%) 3.6323 7.34
0m 23s (- 0m 3s) (4400 88%) 3.8263 7.64
0m 23s (- 0m 2s) (4500 90%) 3.5854 7.44
0m 24s (- 0m 2s) (4600 92%) 3.6472 7.30
0m 24s (- 0m 1s) (4700 94%) 3.4531 6.62
0m 25s (- 0m 1s) (4800 96%) 3.6653 7.36
0m 25s (- 0m 0s) (4900 98%) 3.6198 7.28
0m 26s (- 0m 0s) (5000 100%) 3.8120 7.42
0m 0s (- 0m 31s) (100 2%) 3.7274 8.03
0m 1s (- 0m 30s) (200 4%) 3.5048 7.06
0m 1s (- 0m 29s) (300 6%) 3.5894 7.30
0m 2s (- 0m 27s) (400 8%) 3.3555 6.86
0m 3s (- 0m 27s) (500 10%) 3.6877 7.49
0m 3s (- 0m 27s) (600 12%) 3.7644 7.48
0m 4s (- 0m 26s) (700 14%) 3.5439 7.06
0m 4s (- 0m 26s) (800 16%) 4.0254 7.88
0m 5s (- 0m 24s) (900 18%) 3.5609 7.37
0m 6s (- 0m 24s) (1000 20%) 3.7567 7.49
0m 6s (- 0m 23s) (1100 22%) 3.6788 7.58
0m 7s (- 0m 22s) (1200 24%) 3.5866 7.30
0m 7s (- 0m 22s) (1300 26%) 3.7863 7.75
0m 8s (- 0m 21s) (1400 28%) 3.7586 7.41
0m 8s (- 0m 

0m 21s (- 0m 2s) (4500 90%) 3.4318 7.11
0m 22s (- 0m 1s) (4600 92%) 3.8793 7.87
0m 22s (- 0m 1s) (4700 94%) 3.9009 8.01
0m 23s (- 0m 0s) (4800 96%) 3.7366 7.66
0m 23s (- 0m 0s) (4900 98%) 3.6211 7.42
0m 24s (- 0m 0s) (5000 100%) 3.7660 7.55
0m 0s (- 0m 23s) (100 2%) 3.3664 7.06
0m 0s (- 0m 23s) (200 4%) 3.9198 7.57
0m 1s (- 0m 22s) (300 6%) 3.5387 6.93
0m 1s (- 0m 21s) (400 8%) 3.6943 7.55
0m 2s (- 0m 21s) (500 10%) 3.5564 7.47
0m 2s (- 0m 20s) (600 12%) 3.8122 7.80
0m 3s (- 0m 20s) (700 14%) 3.5455 7.40
0m 3s (- 0m 19s) (800 16%) 3.4197 6.94
0m 4s (- 0m 19s) (900 18%) 3.6127 7.31
0m 4s (- 0m 18s) (1000 20%) 3.7678 7.65
0m 5s (- 0m 18s) (1100 22%) 3.7676 7.64
0m 5s (- 0m 18s) (1200 24%) 3.5832 7.27
0m 6s (- 0m 17s) (1300 26%) 3.8684 8.06
0m 6s (- 0m 17s) (1400 28%) 3.6287 7.55
0m 7s (- 0m 16s) (1500 30%) 3.6009 7.18
0m 7s (- 0m 16s) (1600 32%) 3.5517 7.05
0m 8s (- 0m 15s) (1700 34%) 3.3168 6.96
0m 8s (- 0m 15s) (1800 36%) 4.1436 7.89
0m 9s (- 0m 14s) (1900 38%) 3.6856 7.26
0m 9s (- 0m 

0m 26s (- 0m 0s) (5000 100%) 3.5623 7.49
0m 0s (- 0m 23s) (100 2%) 3.6847 7.50
0m 0s (- 0m 23s) (200 4%) 3.7571 7.57
0m 1s (- 0m 23s) (300 6%) 3.6961 7.63
0m 1s (- 0m 22s) (400 8%) 3.6184 7.30
0m 2s (- 0m 23s) (500 10%) 3.8745 7.70
0m 3s (- 0m 22s) (600 12%) 3.5855 7.09
0m 3s (- 0m 22s) (700 14%) 3.6325 7.52
0m 4s (- 0m 21s) (800 16%) 3.5979 7.23
0m 4s (- 0m 20s) (900 18%) 3.7526 7.28
0m 5s (- 0m 20s) (1000 20%) 3.5998 7.10
0m 5s (- 0m 19s) (1100 22%) 3.9905 8.02
0m 6s (- 0m 19s) (1200 24%) 3.5715 7.34
0m 6s (- 0m 18s) (1300 26%) 3.7272 7.64
0m 7s (- 0m 18s) (1400 28%) 3.6764 7.40
0m 7s (- 0m 18s) (1500 30%) 3.9377 7.63
0m 8s (- 0m 17s) (1600 32%) 3.5365 7.22
0m 8s (- 0m 17s) (1700 34%) 3.6799 7.38
0m 9s (- 0m 17s) (1800 36%) 3.3782 7.17
0m 10s (- 0m 16s) (1900 38%) 3.4938 7.13
0m 10s (- 0m 16s) (2000 40%) 3.6105 7.11
0m 11s (- 0m 15s) (2100 42%) 3.3589 7.25
0m 11s (- 0m 15s) (2200 44%) 3.5112 7.00
0m 12s (- 0m 14s) (2300 46%) 3.4606 7.03
0m 12s (- 0m 13s) (2400 48%) 3.4769 7.18
0m 13s

In [ ]:
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.01)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.005)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.0025)
#word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=0.001)

In [23]:
torch.save(skipgram.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_skipgram_100.pth')

# 3  Sentence Embedding Pretraining

In [ ]:
from chatNLP.modules.Encoder_Words_Recurrent import RecurrentWordsEncoder
from chatNLP.modules.Decoder_Words           import WordsDecoder
from chatNLP.modules.Decoder_Words_Attn      import AttnWordsDecoder

## 3.1 Data

In [ ]:
clean_sentences = [sentence for sentence in list(set([' '.join(s) for diag in dialogues_Master for pair in diag for s in pair]))]
clean_sentences = [s.split() for s in clean_sentences]
print(len(clean_sentences))
print(max([len(s) for s in clean_sentences]))

In [ ]:
clean_sentences_var = [variableFromSentence(lang_M2DS, s, rand = 0) for s in clean_sentences]

In [ ]:
clean_sentences_var_025 = [variableFromSentence(lang_M2DS, s, rand = 0.25) for s in clean_sentences]

## 3.2 Model

In [ ]:
class AutoEncoderTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD, 
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, encoder, decoder, sentence, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        sentence_length = len(sentence)
        sentence = sentence.to(self.device)
        word_hiddens, last_hidden = encoder(sentence)
        output_sentence, log_probas = decoder(query_words     = word_hiddens, 
                                              query_vector    = None, 
                                              decision_vector = last_hidden.view(1, 1, -1), 
                                              target_answer   = sentence)

        loss, loss_diff_mots = self.distance(log_probas, sentence)        
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), self.clip)
        _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), self.clip)
        optimizer.step()
        return loss.data[0] / sentence_length , loss_diff_mots
        
        
    def train(self, encoder, decoder, sentences, n_iters = 10000, learning_rate = 0.01, dic = None, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        start = time.time()
        optimizer = self.optimizer([param for param in encoder.parameters() if param.requires_grad == True] + 
                                   [param for param in decoder.parameters() if param.requires_grad == True], 
                                   lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if return_errors : errors = []
        for iter in range(1, n_iters + 1):
            j = int(random.choice(dic)) if dic is not None else int(random.choice(range(len(sentences))))
            sentence = sentences[j]
            loss, loss_diff_mots = self.trainLoop(encoder, decoder, sentence, optimizer)
            if return_errors and loss_diff_mots > 0 and j not in errors : errors.append(j)
            # quantité d'erreurs sur la réponse i
            print_loss_total += loss
            print_loss_diff_mots_total += loss_diff_mots       
            if iter % self.print_every == 0:
                print_loss_avg = print_loss_total / self.print_every
                print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                print_loss_total = 0
                print_loss_diff_mots_total = 0
                print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg, print_loss_diff_mots_avg))
        if return_errors : return errors

In [ ]:
embedding = skipgram.embedding
for param in embedding.parameters() : param.requires_grad = False


encoder = RecurrentWordsEncoder(device, 
                                embedding = embedding, 
                                hidden_dim = 100, 
                                n_layers = 1, 
                                dropout = 0.15)


decoder = AttnWordsDecoder(device,
                           embedding = embedding,                                   
                           hidden_dim = encoder.output_dim,
                           n_layers = 1,
                           tracking_dim = 0,
                           dropout = 0.15,
                           tf_ratio = 1.0,
                           bound = 75) 


encoder = encoder.to(device)
decoder = decoder.to(device)

In [ ]:
autoencoder_trainer = AutoEncoderTrainer(device = device,
                                         criterion = nn.NLLLoss(), 
                                         optimizer = optim.SGD, 
                                         clipping = 10,
                                         print_every = 100)

## 3.3 Training

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.05)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.02)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.005)

In [ ]:
encoder.train()
decoder.train()
autoencoder_trainer.train(encoder, decoder, sentences = clean_sentences_var, n_iters = 10000, learning_rate = 0.005)

In [ ]:
torch.save(encoder.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_encoder_150.pth')
torch.save(decoder.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_decoder_150.pth')

***
# 4 + 5 + 6 Chatbot implementation

In [17]:
from chatNLP.models.Chatbot import Chatbot, CreateBot, BotTrainer

## 6.1 Data

In [ ]:
dialogues_Master_var = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0)

In [26]:
dialogues_Master_test_var = variableFromAllDialogues(lang_M2DS, dialogues_Master_test, rand = 0)

In [ ]:
bound = max([max([len(el[1]) for el in dial]) for dial in dialogues_Master_var])
bound

## 6.2 Model

In [18]:
# entrainement avec balayage de chaque dialogue en entier 

class BotTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD, 
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, agent, dialogue, target_answer, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        query = dialogue[-1][0].to(self.device)
        target_answer = target_answer.to(self.device)
        answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue, target_answer) 
        loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(agent.parameters(), self.clip)
        optimizer.step()
        return loss.data[0] / len(target_answer), loss_diff_mots
        
        
    def train(self, agent, dialogues, n_iters = 10000, learning_rate=0.01, dic = None, per_dialogue = False, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        if type(dialogues[0][0]) == list :
            debut = 0
            double = True
        else :
            debut = 1
            double = False
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if return_errors : errors = {}
        iter = 1
        while iter < n_iters :
            if dic is not None :
                j = int(random.choice(list(dic.keys())))
                training_dialogue = dialogues[j]
                i = random.choice(dic[j])
                partie_dialogue = training_dialogue[:i+1-debut]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots 
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))
            elif per_dialogue :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                indices = list(range(debut, len(training_dialogue)))
                random.shuffle(indices)
                for i in indices :
                    partie_dialogue = training_dialogue[:i+1]
                    target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                    loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                    if return_errors and loss_diff_mots > 0 :
                        if j not in list(errors.keys()) : errors[j] = []
                        if i not in errors[j] : errors[j].append(i)
                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss
                    print_loss_diff_mots_total += loss_diff_mots
                    iter += 1
                    if iter % self.print_every == 0:
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                      print_loss_avg, print_loss_diff_mots_avg))
            else :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                i = random.choice(range(debut, len(training_dialogue)))
                partie_dialogue = training_dialogue[:i+1]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
                loss, loss_diff_mots = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
                if return_errors and loss_diff_mots > 0 :
                    if j not in list(errors.keys()) : errors[j] = []
                    if i not in errors[j] : errors[j].append(i)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots
                iter += 1
                if iter % self.print_every == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                                  print_loss_avg, print_loss_diff_mots_avg))


        if return_errors : return errors
                
                
    def ErrorCount(self, agent, dialogues):
        bound = 10
        ERRORS = [0 for i in range(bound +1)]
        repartitionError = {}
        for i in range(bound +1) :
            repartitionError[i] = []
        liste = []
        for k, input_dialogue in enumerate(dialogues):
            for l in range(len(input_dialogue)):
                if len(input_dialogue[l][1])>0 :
                    dialogue = input_dialogue[:l+1]
                    #target_answer = variableFromSentence(agent.output_lang, input_dialogue[l][1])
                    target_answer = input_dialogue[l][1]
                    target_answer = target_answer.to(self.device)
                    answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue)
                    loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                    if loss_diff_mots > bound :
                        ERRORS = ERRORS + [0 for i in range(loss_diff_mots - bound)]
                        for i in range(bound +1, loss_diff_mots +1) :
                            repartitionError[i] = []
                        bound  = loss_diff_mots
                    ERRORS[loss_diff_mots] += 1
                    if loss_diff_mots > 0 :
                        liste.append([k, l, loss_diff_mots])
        for triple in liste:
            repartitionError[triple[2]].append(triple[:2])
        print("The repartition of errors :", ERRORS)
        return repartitionError


    def DialoguesWithErrors(self, agent, dialogues) :
        '''Returns a dictionnary, with indices of dialogues and index of line in dialogue
           where a mistake was made.
        '''
        start = time.time()
        Sortie = {}
        L = len(dialogues)
        for i, dialogue in enumerate(dialogues) :
            errs = []
            for j in range(len(dialogue)) :
                target_answer = dialogue[j][1]
                target_answer = target_answer.to(self.device)
                answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue[:j+1], target_answer)
                loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                if loss_diff_mots > 0 :
                    errs.append(j)
            if errs != []:
                Sortie[i] = errs
            if (i+1) % self.print_every == 0:
                print('%s (%d %d%%)' % (self.timeSince(start, (i+1) / L),
                                             (i+1), (i+1) / L * 100))
        return Sortie

In [28]:
# entrainement avec auteoencodage
class BotTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(), 
                 optimizer = optim.SGD,
                 autoencoding_ratio = 0.5,
                 clipping = 10, 
                 print_every=100):
        
        # relevant quantities
        self.device = device
        self.criterion = criterion.to(device)
        self.optimizer = optimizer
        self.aa = autoencoding_ratio
        self.clip = clipping
        self.print_every = print_every# timer
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def distance(self, agent_outputs, target_answer) :
        """ Compute cumulated error between predicted output and ground answer."""
        loss = 0
        loss_diff_mots = 0
        agent_outputs_length = len(agent_outputs)
        target_length = len(target_answer)
        Max = max(agent_outputs_length, target_length)
        Min = min(agent_outputs_length, target_length)   
        for i in range(Min):
            agent_output = agent_outputs[i]
            target_word = target_answer[i]
            #print(agent_output.size(), target_answer.size())
            factor = (1 + Max - Min) if i == Min -1 else 1
            loss += factor * self.criterion(agent_output, target_word)
            topv, topi = agent_output.data.topk(1)
            ni = topi[0][0]
            if ni != target_word.data[0]:
                loss_diff_mots += 1
        loss_diff_mots += Max - Min
        return loss, loss_diff_mots
        
        
    def trainLoop(self, agent, dialogue, target_answer, optimizer):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        query = dialogue[-1][0].to(self.device)
        target_answer = target_answer.to(self.device)
        answer, agent_outputs, attn1_w, attn2_w, autoencoder_outputs = agent.answerTrain(dialogue, target_answer) 
        loss1, loss_diff_mots1 = self.distance(agent_outputs, target_answer)       
        loss2, loss_diff_mots2 = self.distance(autoencoder_outputs, query) if autoencoder_outputs is not None else None, None
        loss = loss1 + self.aa * loss2 autoencoder_outputs is not None else loss1
        loss.backward()
        _ = torch.nn.utils.clip_grad_norm_(agent.parameters(), self.clip)
        optimizer.step()
        l1 = loss1.data[0] / len(target_answer)
        l2 = loss2.data[0] / len(query) if autoencoder_outputs is not None else None
        return l1, loss_diff_mots1, l2 , loss_diff_mots2
        
        
    def train(self, agent, dialogues, n_iters = 10000, learning_rate=0.01, dic = None, return_errors = False):
        """Performs training over a given dataset and along a specified amount of loops."""
        if type(dialogues[0][0]) == list :
            debut = 0
            double = True
        else :
            debut = 1
            double = False
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss1_total = 0
        print_loss2_total = 0
        print_loss_diff_mots1_total = 0
        print_loss_diff_mots2_total = 0
        if return_errors : errors = {}
        for iter in range(1, n_iters + 1):
            if dic is not None :
                j = int(random.choice(list(dic.keys())))
                training_dialogue = dialogues[j]
                i = random.choice(dic[j])
                partie_dialogue = training_dialogue[:i+1-debut]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]
            else :
                j = int(random.choice(range(len(dialogues))))
                training_dialogue = dialogues[j]
                i = random.choice(range(debut, len(training_dialogue)))
                partie_dialogue = training_dialogue[:i+1]
                target_answer = training_dialogue[i][1] if double else training_dialogue[i]

            loss1, loss_diff_mots1, loss2, loss_diff_mots2 = self.trainLoop(agent, partie_dialogue, target_answer, optimizer)
            if return_errors and loss_diff_mots1 > 0 :
                if j not in list(errors.keys()) : errors[j] = []
                if i not in errors[j] : errors[j].append(i)
            # quantité d'erreurs sur la réponse i
            print_loss1_total += loss1
            print_loss2_total += loss2
            print_loss_diff_mots1_total += loss_diff_mots1  
            print_loss_diff_mots2_total += loss_diff_mots2  
            if iter % self.print_every == 0:
                print_loss1_avg = print_loss1_total / self.print_every
                print_loss2_avg = print_loss2_total / self.print_every
                print_loss_diff_mots1_avg = print_loss_diff_mots1_total / self.print_every
                print_loss_diff_mots2_avg = print_loss_diff_mots2_total / self.print_every
                print_loss1_total = 0
                print_loss2_total = 0
                print_loss_diff_mots1_total = 0
                print_loss_diff_mots2_total = 0
                print('%s (%d %d%%) %.4f %.2f | %.4f %.2f' % (self.timeSince(start, iter / n_iters), iter, iter / n_iters * 100, 
                                                              print_loss1_avg, print_loss_diff_mots1_avg,
                                                              print_loss2_avg, print_loss_diff_mots2_avg))
        if return_errors : return errors
                
                
    def ErrorCount(self, agent, dialogues):
        bound = 10
        ERRORS = [0 for i in range(bound +1)]
        repartitionError = {}
        for i in range(bound +1) :
            repartitionError[i] = []
        liste = []
        for k, input_dialogue in enumerate(dialogues):
            for l in range(len(input_dialogue)):
                if len(input_dialogue[l][1])>0 :
                    dialogue = input_dialogue[:l+1]
                    #target_answer = variableFromSentence(agent.output_lang, input_dialogue[l][1])
                    target_answer = input_dialogue[l][1]
                    target_answer = target_answer.to(self.device)
                    answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue)
                    loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                    if loss_diff_mots > bound :
                        ERRORS = ERRORS + [0 for i in range(loss_diff_mots - bound)]
                        for i in range(bound +1, loss_diff_mots +1) :
                            repartitionError[i] = []
                        bound  = loss_diff_mots
                    ERRORS[loss_diff_mots] += 1
                    if loss_diff_mots > 0 :
                        liste.append([k, l, loss_diff_mots])
        for triple in liste:
            repartitionError[triple[2]].append(triple[:2])
        print("The repartition of errors :", ERRORS)
        return repartitionError


    def DialoguesWithErrors(self, agent, dialogues) :
        '''Returns a dictionnary, with indices of dialogues and index of line in dialogue
           where a mistake was made.
        '''
        start = time.time()
        Sortie = {}
        L = len(dialogues)
        for i, dialogue in enumerate(dialogues) :
            errs = []
            for j in range(len(dialogue)) :
                target_answer = dialogue[j][1]
                target_answer = target_answer.to(self.device)
                answer, agent_outputs, attn1_w, attn2_w, _ = agent.answerTrain(dialogue[:j+1], target_answer)
                loss, loss_diff_mots = self.distance(agent_outputs, target_answer)
                if loss_diff_mots > 0 :
                    errs.append(j)
            if errs != []:
                Sortie[i] = errs
            if (i+1) % self.print_every == 0:
                print('%s (%d %d%%)' % (self.timeSince(start, (i+1) / L),
                                             (i+1), (i+1) / L * 100))
        return Sortie

In [19]:
trainer = BotTrainer(device = device,
                     criterion = nn.NLLLoss(), 
                     optimizer = optim.SGD,
                     #autoencoding_ratio = 0.5,
                     clipping = 10,
                     print_every = 100)

In [20]:
# query_vector <- decision_vector en WordDecoder

chatbot = CreateBot(  lang_M2DS,
                      embedding = skipgram.embedding,
                      hidden_dim = 100,
                      n_layers = 1, # doit être gardé à 1
                    
                      sentence_hidden_dim = 50,
                      hops = 3,
                      share = True, #False
                      transf = True,
                      dropout = 0.15,
                    
                      attn_decoder_n_layers = -1, # -1 : pas d'attention durant le décodage
                      language_model_n_layers = 0,
                      tf_ratio = 1.0,
                      bound = 75,
                      autoencoding = False,
                    
                      device = device
                     )

chatbot.nbParametres()

888091

In [23]:
chatbot.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_8.pth')) # chatbot_original / 26 / 39

In [24]:
chatbot.decoder.tf_ratio = 0.25

In [26]:
# si on veut voir le pipeline du modèle
chatbot

Chatbot(
  (encoder): RecurrentWordsEncoder(
    (embedding): Embedding(1291, 100)
    (dropout): Dropout(p=0.15)
    (bigru): GRU(100, 100, bidirectional=True)
  )
  (attention): RecurrentHierarchicalAttention(
    (dropout): Dropout(p=0.15)
    (attn1): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=400, out_features=200, bias=True)
        (attn_v): Linear(in_features=200, out_features=1, bias=False)
      )
      (1): AdditiveAttention(
        (attn_layer): Linear(in_features=400, out_features=200, bias=True)
        (attn_v): Linear(in_features=200, out_features=1, bias=False)
      )
      (2): AdditiveAttention(
        (attn_layer): Linear(in_features=400, out_features=200, bias=True)
        (attn_v): Linear(in_features=200, out_features=1, bias=False)
      )
    )
    (bigru): GRU(200, 50, bidirectional=True)
    (attn2): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=100, bias=True)
 

## 6.3 Training

### 6.3.1 Standard training

In [27]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 21s (- 70m 3s) (100 0%) 5.9916 15.43
0m 44s (- 74m 9s) (200 1%) 5.2349 14.83
1m 7s (- 73m 44s) (300 1%) 4.1906 11.11
1m 25s (- 69m 49s) (400 2%) 3.5935 10.49
1m 42s (- 66m 38s) (500 2%) 3.4533 10.46
2m 5s (- 67m 43s) (600 3%) 3.3018 9.04
2m 23s (- 65m 54s) (700 3%) 2.9122 8.37
2m 44s (- 65m 45s) (800 4%) 3.0128 9.85
3m 9s (- 66m 54s) (900 4%) 2.7085 8.15
3m 32s (- 67m 15s) (1000 5%) 2.3228 8.11
3m 56s (- 67m 41s) (1100 5%) 2.6188 7.53
4m 17s (- 67m 19s) (1200 6%) 1.9515 5.80
4m 35s (- 65m 59s) (1300 6%) 1.8923 6.63
4m 57s (- 65m 57s) (1400 7%) 2.2244 7.15
5m 21s (- 66m 6s) (1500 7%) 2.0422 7.39
5m 41s (- 65m 29s) (1600 8%) 1.9793 6.51
6m 0s (- 64m 41s) (1700 8%) 2.0284 7.70
6m 21s (- 64m 21s) (1800 9%) 2.0020 7.46
6m 43s (- 64m 1s) (1900 9%) 2.0394 6.62
7m 5s (- 63m 50s) (2000 10%) 1.8896 6.69
7m 25s (- 63m 13s) (2100 10%) 1.7020 7.07
7m 42s (- 62m 22s) (2200 11%) 1.5183 5.97
8m 3s (- 61m 58s) (2300 11%) 1.5136 5.51
8m 24s (- 61m 39s) (2400 12%) 1.6101 5.57
8m 44s (- 61m 8s) (2500 1

76m 48s (- 2m 47s) (19300 96%) 0.1423 0.46
77m 10s (- 2m 23s) (19400 97%) 0.1085 0.32
77m 28s (- 1m 59s) (19500 97%) 0.1398 0.54
77m 49s (- 1m 35s) (19600 98%) 0.2915 1.68
78m 11s (- 1m 11s) (19700 98%) 0.2017 1.57
78m 31s (- 0m 47s) (19800 99%) 0.1594 0.90
78m 49s (- 0m 23s) (19900 99%) 0.1809 0.88
79m 13s (- 0m 0s) (20000 100%) 0.1119 0.36


In [28]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_1.pth')

In [29]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 21s (- 72m 19s) (100 0%) 0.1399 0.57
0m 41s (- 67m 44s) (200 1%) 0.1981 0.76
1m 2s (- 68m 27s) (300 1%) 0.1439 0.85
1m 22s (- 67m 43s) (400 2%) 0.1212 0.53
1m 45s (- 68m 37s) (500 2%) 0.1458 0.69
2m 4s (- 67m 18s) (600 3%) 0.1135 0.64
2m 29s (- 68m 35s) (700 3%) 0.1328 0.37
2m 51s (- 68m 28s) (800 4%) 0.1332 0.66
3m 13s (- 68m 30s) (900 4%) 0.1079 0.38
3m 37s (- 68m 54s) (1000 5%) 0.2059 0.93
3m 56s (- 67m 47s) (1100 5%) 0.2344 0.79
4m 15s (- 66m 40s) (1200 6%) 0.1939 0.92
4m 33s (- 65m 39s) (1300 6%) 0.1308 0.33
4m 56s (- 65m 34s) (1400 7%) 0.1347 0.49
5m 15s (- 64m 56s) (1500 7%) 0.1115 0.38
5m 38s (- 64m 47s) (1600 8%) 0.1168 0.51
5m 57s (- 64m 12s) (1700 8%) 0.1934 0.94
6m 16s (- 63m 29s) (1800 9%) 0.1583 0.96
6m 36s (- 62m 54s) (1900 9%) 0.1305 0.58
6m 59s (- 62m 57s) (2000 10%) 0.1354 0.54
7m 22s (- 62m 54s) (2100 10%) 0.1066 0.29
7m 42s (- 62m 24s) (2200 11%) 0.1619 0.78
8m 6s (- 62m 22s) (2300 11%) 0.0886 0.45
8m 24s (- 61m 40s) (2400 12%) 0.0833 0.31
8m 44s (- 61m 12s) (250

65m 45s (- 2m 23s) (19300 96%) 0.0659 0.71
66m 2s (- 2m 2s) (19400 97%) 0.0652 0.20
66m 23s (- 1m 42s) (19500 97%) 0.0717 0.26
66m 42s (- 1m 21s) (19600 98%) 0.0634 0.35
67m 3s (- 1m 1s) (19700 98%) 0.0468 0.13
67m 23s (- 0m 40s) (19800 99%) 0.0690 0.29
67m 43s (- 0m 20s) (19900 99%) 0.0679 0.34
68m 2s (- 0m 0s) (20000 100%) 0.0703 0.24


In [30]:
# 9599
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_2.pth')

In [31]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [9599, 46, 57, 1, 0, 61, 113, 176, 92, 247, 513, 1333, 646, 802, 1286, 575, 934, 1218, 637, 512, 151, 239, 3, 32, 183, 41, 143, 7, 35, 13, 0, 0, 0, 2, 0, 16, 3, 0, 0, 0, 0, 0, 0, 5, 2, 0, 20, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64, 0, 0, 0, 0, 0, 0, 0, 0, 35]


In [32]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.05, dic = errors)

0m 18s (- 14m 57s) (100 2%) 0.1094 0.55
0m 37s (- 15m 8s) (200 4%) 0.1273 0.62
0m 56s (- 14m 38s) (300 6%) 0.1672 0.86
1m 17s (- 14m 48s) (400 8%) 0.1037 0.57
1m 36s (- 14m 32s) (500 10%) 0.1489 0.67
1m 57s (- 14m 19s) (600 12%) 0.1314 0.64
2m 17s (- 14m 3s) (700 14%) 0.1400 0.50
2m 35s (- 13m 36s) (800 16%) 0.1346 0.57
2m 53s (- 13m 12s) (900 18%) 0.1346 0.53
3m 14s (- 12m 57s) (1000 20%) 0.1136 0.52
3m 32s (- 12m 33s) (1100 22%) 0.1239 0.56
3m 50s (- 12m 11s) (1200 24%) 0.1323 0.52
4m 10s (- 11m 51s) (1300 26%) 0.1252 0.68
4m 30s (- 11m 34s) (1400 28%) 0.1052 0.45
4m 49s (- 11m 14s) (1500 30%) 0.1068 0.40
5m 6s (- 10m 52s) (1600 32%) 0.1245 0.55
5m 25s (- 10m 31s) (1700 34%) 0.1074 0.41
5m 43s (- 10m 9s) (1800 36%) 0.1077 0.36
6m 1s (- 9m 50s) (1900 38%) 0.0912 0.27
6m 20s (- 9m 31s) (2000 40%) 0.1183 0.53
6m 40s (- 9m 13s) (2100 42%) 0.0721 0.30
6m 59s (- 8m 53s) (2200 44%) 0.1194 0.46
7m 18s (- 8m 34s) (2300 46%) 0.1110 0.53
7m 37s (- 8m 15s) (2400 48%) 0.0852 0.33
7m 56s (- 7m 56s

### 6.3.2 Teacher Forcing Ratio = 0.5

In [33]:
chatbot.decoder.tf_ratio = 0.5

In [34]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 19s (- 64m 42s) (100 0%) 0.7961 0.60
0m 38s (- 63m 4s) (200 1%) 1.6469 1.37
0m 59s (- 65m 3s) (300 1%) 1.4252 1.44
1m 21s (- 66m 24s) (400 2%) 1.2682 1.19
1m 42s (- 66m 36s) (500 2%) 1.0677 1.36
2m 1s (- 65m 28s) (600 3%) 1.3275 1.12
2m 21s (- 64m 49s) (700 3%) 1.8297 1.83
2m 38s (- 63m 30s) (800 4%) 1.2056 0.94
3m 1s (- 64m 9s) (900 4%) 1.1803 1.81
3m 20s (- 63m 25s) (1000 5%) 0.5553 1.04
3m 39s (- 62m 47s) (1100 5%) 1.6608 1.77
4m 2s (- 63m 12s) (1200 6%) 0.7305 1.00
4m 19s (- 62m 17s) (1300 6%) 1.5675 1.14
4m 36s (- 61m 10s) (1400 7%) 1.9259 2.06
4m 54s (- 60m 37s) (1500 7%) 0.1997 0.57
5m 16s (- 60m 43s) (1600 8%) 1.0881 1.30
5m 36s (- 60m 26s) (1700 8%) 1.3278 2.14
5m 58s (- 60m 20s) (1800 9%) 0.7759 0.90
6m 15s (- 59m 33s) (1900 9%) 0.4660 0.55
6m 34s (- 59m 12s) (2000 10%) 1.3657 1.39
6m 52s (- 58m 36s) (2100 10%) 0.5425 0.66
7m 13s (- 58m 23s) (2200 11%) 0.6043 0.60
7m 33s (- 58m 12s) (2300 11%) 1.0258 1.02
7m 53s (- 57m 51s) (2400 12%) 0.3951 0.46
8m 13s (- 57m 31s) (2500 1

67m 6s (- 2m 26s) (19300 96%) 1.0490 0.50
67m 22s (- 2m 5s) (19400 97%) 0.2035 0.32
67m 46s (- 1m 44s) (19500 97%) 0.5531 0.84
68m 6s (- 1m 23s) (19600 98%) 0.6228 0.61
68m 35s (- 1m 2s) (19700 98%) 0.8836 0.95
68m 59s (- 0m 41s) (19800 99%) 0.7049 0.79
69m 21s (- 0m 20s) (19900 99%) 0.3600 0.38
69m 47s (- 0m 0s) (20000 100%) 0.2391 0.32


In [35]:
# 10041
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_3.pth')

In [36]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10041, 18, 85, 3, 3, 82, 70, 193, 172, 112, 440, 1044, 754, 515, 1142, 600, 758, 1538, 925, 420, 171, 250, 3, 106, 60, 93, 84, 8, 15, 79, 0, 0, 0, 0, 0, 17, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 35]


In [37]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.05, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 35s (- 29m 22s) (100 2%) 2.2717 3.04
1m 9s (- 27m 49s) (200 4%) 0.9592 1.57
1m 42s (- 26m 41s) (300 6%) 1.2794 2.07
2m 12s (- 25m 27s) (400 8%) 1.5005 1.67
2m 45s (- 24m 51s) (500 10%) 1.1550 1.57
3m 18s (- 24m 13s) (600 12%) 2.0762 3.88
3m 53s (- 23m 57s) (700 14%) 1.6810 2.56
4m 24s (- 23m 8s) (800 16%) 2.0607 2.88
4m 53s (- 22m 18s) (900 18%) 1.9023 2.95
5m 26s (- 21m 45s) (1000 20%) 1.2751 2.10
5m 56s (- 21m 3s) (1100 22%) 1.0383 1.26
6m 27s (- 20m 26s) (1200 24%) 0.8791 1.46
6m 57s (- 19m 47s) (1300 26%) 1.2507 2.20
7m 29s (- 19m 17s) (1400 28%) 0.8046 1.58
8m 2s (- 18m 44s) (1500 30%) 0.9844 1.61
8m 35s (- 18m 16s) (1600 32%) 1.3697 2.05
9m 8s (- 17m 44s) (1700 34%) 1.7874 3.03
9m 39s (- 17m 9s) (1800 36%) 0.6789 1.17
10m 10s (- 16m 36s) (1900 38%) 1.3622 1.78
10m 39s (- 15m 59s) (2000 40%) 1.3190 2.67
11m 11s (- 15m 26s) (2100 42%) 1.4528 3.89
11m 41s (- 14m 52s) (2200 44%) 1.7965 2.92
12m 15s (- 14m 23s) (2300 46%) 0.9981 2.18
12m 45s (- 13m 49s) (2400 48%) 1.0689 1.92
13m 1

83m 39s (- 32m 32s) (14400 72%) 0.3855 0.45
84m 15s (- 31m 57s) (14500 72%) 0.4808 0.76
84m 47s (- 31m 21s) (14600 73%) 0.3793 0.47
85m 15s (- 30m 44s) (14700 73%) 0.2193 0.33
85m 45s (- 30m 7s) (14800 74%) 0.3248 0.34
86m 20s (- 29m 33s) (14900 74%) 0.3845 1.02
86m 58s (- 28m 59s) (15000 75%) 0.5780 0.70
87m 27s (- 28m 22s) (15100 75%) 0.3960 0.47
88m 9s (- 27m 50s) (15200 76%) 0.5031 0.51
88m 48s (- 27m 16s) (15300 76%) 0.1562 0.31
89m 22s (- 26m 41s) (15400 77%) 0.2326 0.22
89m 59s (- 26m 7s) (15500 77%) 0.9244 1.52
90m 32s (- 25m 32s) (15600 78%) 0.2381 0.34
91m 0s (- 24m 55s) (15700 78%) 0.2399 0.32
91m 31s (- 24m 19s) (15800 79%) 0.4761 0.67
92m 0s (- 23m 43s) (15900 79%) 0.2819 0.41
92m 40s (- 23m 10s) (16000 80%) 0.0566 0.16
93m 10s (- 22m 34s) (16100 80%) 0.2200 0.18
93m 45s (- 21m 59s) (16200 81%) 0.1992 0.29
94m 20s (- 21m 24s) (16300 81%) 0.1288 0.17
94m 56s (- 20m 50s) (16400 82%) 0.2694 0.31
95m 36s (- 20m 16s) (16500 82%) 0.4358 0.65
96m 12s (- 19m 42s) (16600 83%) 0.439

In [38]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_4.pth')

### 6.3.3 Random masking 25 % of corpus words

In [39]:
dialogues_Master_var_025 = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0.25)

In [40]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.05, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.05, return_errors = True)

0m 53s (- 43m 59s) (100 2%) 1.1496 2.21
1m 34s (- 37m 51s) (200 4%) 0.4267 0.91
2m 10s (- 33m 58s) (300 6%) 1.3889 2.78
2m 43s (- 31m 23s) (400 8%) 1.5036 2.21
3m 18s (- 29m 46s) (500 10%) 1.4051 2.05
3m 55s (- 28m 45s) (600 12%) 1.6844 3.06
4m 31s (- 27m 48s) (700 14%) 1.0858 1.77
5m 6s (- 26m 49s) (800 16%) 0.7223 1.18
5m 41s (- 25m 56s) (900 18%) 2.1269 2.48
6m 18s (- 25m 12s) (1000 20%) 0.4699 1.07
6m 52s (- 24m 22s) (1100 22%) 0.9534 1.84
7m 30s (- 23m 46s) (1200 24%) 1.0837 1.64
8m 6s (- 23m 5s) (1300 26%) 1.0566 1.65
8m 40s (- 22m 18s) (1400 28%) 1.2299 2.74
9m 12s (- 21m 29s) (1500 30%) 1.0713 1.47
9m 48s (- 20m 50s) (1600 32%) 1.2550 1.92
10m 22s (- 20m 8s) (1700 34%) 1.0725 2.29
10m 52s (- 19m 20s) (1800 36%) 0.3281 0.57
11m 24s (- 18m 36s) (1900 38%) 0.6061 0.85
11m 57s (- 17m 56s) (2000 40%) 0.8649 2.02
12m 31s (- 17m 18s) (2100 42%) 0.9898 1.84
13m 4s (- 16m 39s) (2200 44%) 0.6705 1.07
13m 37s (- 16m 0s) (2300 46%) 0.6397 1.81
14m 10s (- 15m 21s) (2400 48%) 0.8468 1.37
14m

76m 49s (- 29m 52s) (14400 72%) 2.6869 4.52
77m 18s (- 29m 19s) (14500 72%) 1.8045 3.61
77m 47s (- 28m 46s) (14600 73%) 2.2370 3.22
78m 27s (- 28m 17s) (14700 73%) 2.0968 3.36
79m 1s (- 27m 45s) (14800 74%) 2.2087 3.20
79m 33s (- 27m 13s) (14900 74%) 1.9974 3.38
79m 56s (- 26m 38s) (15000 75%) 1.2083 2.45
80m 25s (- 26m 5s) (15100 75%) 1.4668 2.32
81m 0s (- 25m 34s) (15200 76%) 1.5155 2.34
81m 33s (- 25m 3s) (15300 76%) 1.7768 3.20
82m 7s (- 24m 31s) (15400 77%) 1.9769 4.27
82m 43s (- 24m 0s) (15500 77%) 1.1664 1.68
83m 13s (- 23m 28s) (15600 78%) 1.4546 2.98
83m 46s (- 22m 56s) (15700 78%) 1.6633 3.34
84m 25s (- 22m 26s) (15800 79%) 1.6296 2.86
84m 56s (- 21m 54s) (15900 79%) 2.9764 4.12
85m 31s (- 21m 22s) (16000 80%) 1.5003 2.60
85m 57s (- 20m 49s) (16100 80%) 2.2540 4.26
86m 33s (- 20m 18s) (16200 81%) 1.5944 2.55
87m 6s (- 19m 46s) (16300 81%) 1.8415 3.13
87m 33s (- 19m 13s) (16400 82%) 1.9389 4.76
88m 0s (- 18m 40s) (16500 82%) 1.2217 1.94
88m 36s (- 18m 8s) (16600 83%) 1.6323 3.

In [41]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_5.pth')

### 6.3.4 Learning rate harved

In [42]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.025, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 38s (- 31m 19s) (100 2%) 0.9885 2.24
1m 15s (- 30m 21s) (200 4%) 1.3203 3.90
1m 53s (- 29m 36s) (300 6%) 1.1371 2.61
2m 31s (- 29m 0s) (400 8%) 1.4253 2.66
3m 9s (- 28m 28s) (500 10%) 0.9745 2.10
3m 47s (- 27m 50s) (600 12%) 1.6260 3.55
4m 24s (- 27m 5s) (700 14%) 0.7619 2.31
4m 56s (- 25m 58s) (800 16%) 0.9040 1.83
5m 30s (- 25m 5s) (900 18%) 1.2267 3.19
6m 1s (- 24m 5s) (1000 20%) 1.0055 1.62
6m 37s (- 23m 27s) (1100 22%) 1.1612 3.17
7m 10s (- 22m 43s) (1200 24%) 0.7062 1.24
7m 44s (- 22m 2s) (1300 26%) 0.1211 0.41
8m 20s (- 21m 27s) (1400 28%) 0.5865 1.63
8m 54s (- 20m 46s) (1500 30%) 0.7803 1.37
9m 25s (- 20m 2s) (1600 32%) 0.8703 2.16
9m 58s (- 19m 22s) (1700 34%) 0.5567 1.48
10m 31s (- 18m 43s) (1800 36%) 0.2596 0.52
11m 2s (- 18m 1s) (1900 38%) 0.3788 0.60
11m 30s (- 17m 15s) (2000 40%) 0.5112 0.81
12m 1s (- 16m 35s) (2100 42%) 1.2792 2.40
12m 31s (- 15m 56s) (2200 44%) 0.7586 2.27
13m 2s (- 15m 19s) (2300 46%) 1.2504 2.11
13m 34s (- 14m 42s) (2400 48%) 0.5008 0.91
14m 4s (- 

77m 15s (- 29m 18s) (14500 72%) 1.4641 2.18
77m 38s (- 28m 43s) (14600 73%) 0.9838 1.80
78m 7s (- 28m 10s) (14700 73%) 1.2872 1.94
78m 40s (- 27m 38s) (14800 74%) 1.3589 2.88
79m 9s (- 27m 5s) (14900 74%) 1.7313 3.39
79m 38s (- 26m 32s) (15000 75%) 2.2457 4.45
80m 5s (- 25m 59s) (15100 75%) 1.5804 1.98
80m 33s (- 25m 26s) (15200 76%) 1.8473 2.92
81m 5s (- 24m 54s) (15300 76%) 1.8689 2.46
81m 32s (- 24m 21s) (15400 77%) 1.8011 2.76
82m 2s (- 23m 49s) (15500 77%) 1.3695 2.51
82m 31s (- 23m 16s) (15600 78%) 1.1669 1.98
82m 57s (- 22m 43s) (15700 78%) 1.5533 3.41
83m 24s (- 22m 10s) (15800 79%) 1.0592 1.96
84m 1s (- 21m 40s) (15900 79%) 1.0275 1.61
84m 35s (- 21m 8s) (16000 80%) 0.6788 1.10
85m 10s (- 20m 37s) (16100 80%) 0.7326 1.26
85m 45s (- 20m 6s) (16200 81%) 1.7921 3.18
86m 21s (- 19m 36s) (16300 81%) 0.6276 1.27
87m 2s (- 19m 6s) (16400 82%) 1.5678 2.89
87m 37s (- 18m 35s) (16500 82%) 1.0463 1.85
88m 7s (- 18m 2s) (16600 83%) 1.8443 2.32
88m 32s (- 17m 29s) (16700 83%) 1.2630 1.52
8

In [43]:
# 12002
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_6.pth') #25

In [44]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12002, 15, 98, 0, 0, 84, 45, 220, 46, 168, 512, 671, 427, 597, 706, 609, 647, 1370, 274, 499, 166, 257, 10, 21, 140, 34, 87, 63, 2, 0, 0, 0, 0, 0, 1, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.3.5 Teacher Forcing Ratio = 0.25

In [45]:
chatbot.decoder.tf_ratio = 0.25

In [46]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 19s (- 63m 42s) (100 0%) 1.9388 3.12
0m 37s (- 62m 27s) (200 1%) 1.6066 3.07
0m 56s (- 61m 22s) (300 1%) 1.5417 2.21
1m 16s (- 62m 28s) (400 2%) 1.3670 2.10
1m 35s (- 62m 18s) (500 2%) 1.5760 2.66
1m 59s (- 64m 12s) (600 3%) 1.2065 2.06
2m 16s (- 62m 48s) (700 3%) 1.9623 3.81
2m 38s (- 63m 22s) (800 4%) 1.7299 2.49
2m 56s (- 62m 36s) (900 4%) 1.7146 3.04
3m 17s (- 62m 27s) (1000 5%) 1.8613 2.85
3m 36s (- 62m 6s) (1100 5%) 1.6473 2.91
3m 57s (- 62m 2s) (1200 6%) 1.7845 3.03
4m 17s (- 61m 50s) (1300 6%) 2.0114 3.25
4m 37s (- 61m 21s) (1400 7%) 1.2691 2.68
4m 54s (- 60m 29s) (1500 7%) 1.7743 3.06
5m 17s (- 60m 51s) (1600 8%) 2.4003 4.35
5m 38s (- 60m 43s) (1700 8%) 2.1696 3.55
5m 56s (- 60m 3s) (1800 9%) 1.2236 2.74
6m 14s (- 59m 23s) (1900 9%) 1.7994 3.57
6m 34s (- 59m 6s) (2000 10%) 2.9431 4.90
6m 52s (- 58m 34s) (2100 10%) 1.4995 2.51
7m 10s (- 58m 3s) (2200 11%) 1.5358 2.12
7m 31s (- 57m 55s) (2300 11%) 1.4026 3.11
7m 48s (- 57m 16s) (2400 12%) 1.3029 2.58
8m 10s (- 57m 10s) (2500 

97m 4s (- 3m 31s) (19300 96%) 2.3484 2.95
97m 33s (- 3m 1s) (19400 97%) 2.1680 3.49
98m 4s (- 2m 30s) (19500 97%) 0.9541 1.48
98m 36s (- 2m 0s) (19600 98%) 1.4974 3.08
99m 11s (- 1m 30s) (19700 98%) 1.7094 2.64
99m 40s (- 1m 0s) (19800 99%) 1.1242 2.55
100m 19s (- 0m 30s) (19900 99%) 1.6133 4.02
100m 48s (- 0m 0s) (20000 100%) 0.8402 1.86


In [47]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_7.pth') #26

In [48]:
# 12281
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12281, 18, 27, 0, 29, 102, 28, 199, 53, 162, 289, 933, 361, 447, 931, 416, 610, 1649, 219, 226, 164, 231, 2, 49, 76, 63, 79, 60, 1, 56, 0, 0, 0, 0, 0, 16, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 35, 1]


In [ ]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.025, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 15000, per_dialogue = True, learning_rate=0.025, return_errors = True)

0m 25s (- 83m 59s) (100 0%) 1.2710 2.72
1m 0s (- 100m 13s) (200 1%) 1.1238 1.81
1m 19s (- 86m 50s) (300 1%) 1.6897 2.79
1m 55s (- 94m 4s) (400 2%) 2.1377 3.77
2m 33s (- 99m 44s) (500 2%) 1.2108 2.13
3m 11s (- 103m 14s) (600 3%) 1.7032 2.61
3m 50s (- 105m 49s) (700 3%) 1.7275 2.78
4m 24s (- 105m 38s) (800 4%) 2.0514 3.81
4m 58s (- 105m 37s) (900 4%) 1.1855 2.50
5m 37s (- 106m 47s) (1000 5%) 1.3432 2.68
6m 12s (- 106m 41s) (1100 5%) 2.0143 3.16
6m 51s (- 107m 34s) (1200 6%) 1.6209 2.81
7m 28s (- 107m 34s) (1300 6%) 0.9512 1.82
8m 3s (- 106m 58s) (1400 7%) 1.6431 3.24
8m 47s (- 108m 24s) (1500 7%) 1.3954 2.96
9m 19s (- 107m 19s) (1600 8%) 2.1645 3.87
9m 50s (- 105m 54s) (1700 8%) 1.3619 2.22
10m 23s (- 105m 9s) (1800 9%) 1.8262 2.93
11m 4s (- 105m 29s) (1900 9%) 1.3356 1.80
11m 39s (- 104m 51s) (2000 10%) 1.6291 2.52
12m 12s (- 104m 0s) (2100 10%) 1.8436 3.75
12m 51s (- 104m 1s) (2200 11%) 1.8586 3.76
13m 19s (- 102m 36s) (2300 11%) 1.4988 2.70
13m 53s (- 101m 53s) (2400 12%) 1.0582 1.68


In [56]:
# 12319
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_100_50_015_8.pth') #26

In [27]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12319, 50, 12, 0, 1, 72, 22, 174, 20, 243, 325, 802, 453, 301, 1032, 366, 560, 1381, 349, 554, 149, 216, 1, 27, 133, 59, 141, 4, 3, 0, 0, 0, 1, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 35, 1]


### 6.3.6 Dropout lowered to 0.1

### TODO

In [56]:
chatbot.train()
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.01, return_errors = True)

0m 20s (- 68m 32s) (100 0%) 0.0040 0.00
0m 41s (- 68m 19s) (200 1%) 0.1132 0.27
1m 5s (- 71m 47s) (300 1%) 0.4854 0.32
1m 21s (- 66m 50s) (400 2%) 0.1007 0.34
1m 45s (- 68m 50s) (500 2%) 0.0801 0.09
2m 9s (- 69m 36s) (600 3%) 0.3116 0.29
2m 30s (- 69m 15s) (700 3%) 0.0046 0.00
2m 53s (- 69m 31s) (800 4%) 0.1297 0.17
3m 14s (- 68m 51s) (900 4%) 0.1852 0.10
3m 36s (- 68m 30s) (1000 5%) 1.4274 1.33
3m 55s (- 67m 34s) (1100 5%) 0.1570 0.13
4m 17s (- 67m 21s) (1200 6%) 0.1084 0.08
4m 37s (- 66m 29s) (1300 6%) 0.4776 0.52
4m 57s (- 65m 53s) (1400 7%) 0.3793 0.36
5m 15s (- 64m 56s) (1500 7%) 0.8786 0.98
5m 34s (- 64m 2s) (1600 8%) 0.0035 0.00
6m 0s (- 64m 43s) (1700 8%) 0.4586 0.76
6m 19s (- 63m 56s) (1800 9%) 0.2471 0.31
6m 37s (- 63m 4s) (1900 9%) 0.0766 0.13
6m 58s (- 62m 49s) (2000 10%) 0.1529 0.13
7m 23s (- 62m 56s) (2100 10%) 0.1646 0.16
7m 42s (- 62m 24s) (2200 11%) 0.0062 0.00
8m 2s (- 61m 54s) (2300 11%) 0.4561 0.49
8m 20s (- 61m 8s) (2400 12%) 0.4994 0.72
8m 41s (- 60m 48s) (2500 12

66m 34s (- 2m 24s) (19300 96%) 0.3252 0.24
66m 53s (- 2m 4s) (19400 97%) 0.2461 0.30
67m 13s (- 1m 43s) (19500 97%) 0.1328 0.14
67m 34s (- 1m 22s) (19600 98%) 0.2014 0.23
67m 56s (- 1m 2s) (19700 98%) 0.0891 0.16
68m 16s (- 0m 41s) (19800 99%) 0.0042 0.01
68m 39s (- 0m 20s) (19900 99%) 0.0153 0.07
68m 56s (- 0m 0s) (20000 100%) 0.0036 0.00


In [57]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_391.pth') #27

In [58]:
# 11421 (h = 100, s = 100) / 10814 (h = 75, s = 50)
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10814, 27, 87, 0, 1, 88, 62, 484, 98, 229, 335, 816, 428, 420, 1265, 926, 768, 846, 478, 418, 341, 283, 0, 5, 151, 171, 65, 69, 0, 44, 54, 0, 0, 0, 1, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31]


In [59]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 5000, learning_rate=0.005, dic = errors)
errors = trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.005, return_errors = True)

0m 30s (- 25m 1s) (100 2%) 1.0448 1.15
0m 56s (- 22m 34s) (200 4%) 1.2816 1.35
1m 22s (- 21m 33s) (300 6%) 0.3364 0.91
1m 47s (- 20m 36s) (400 8%) 0.7546 1.56
2m 10s (- 19m 33s) (500 10%) 0.9986 1.69
2m 32s (- 18m 41s) (600 12%) 0.8717 1.17
2m 53s (- 17m 45s) (700 14%) 0.7318 0.91
3m 13s (- 16m 57s) (800 16%) 0.6875 0.79
3m 32s (- 16m 7s) (900 18%) 1.1511 0.66
3m 52s (- 15m 31s) (1000 20%) 0.5373 0.79
4m 12s (- 14m 55s) (1100 22%) 0.7483 0.73
4m 33s (- 14m 24s) (1200 24%) 0.8407 0.82
4m 51s (- 13m 49s) (1300 26%) 1.5489 2.05
5m 11s (- 13m 19s) (1400 28%) 0.2239 0.26
5m 31s (- 12m 54s) (1500 30%) 0.5032 0.56
5m 52s (- 12m 28s) (1600 32%) 0.5377 0.63
6m 11s (- 12m 1s) (1700 34%) 0.2255 0.26
6m 31s (- 11m 35s) (1800 36%) 0.2125 0.22
6m 51s (- 11m 11s) (1900 38%) 0.6729 0.64
7m 10s (- 10m 45s) (2000 40%) 0.5955 0.64
7m 30s (- 10m 22s) (2100 42%) 0.9905 1.13
7m 51s (- 9m 59s) (2200 44%) 0.6765 0.58
8m 9s (- 9m 34s) (2300 46%) 0.7923 0.86
8m 28s (- 9m 11s) (2400 48%) 0.3983 1.60
8m 49s (- 8m

49m 34s (- 18m 48s) (14500 72%) 0.1984 0.20
49m 56s (- 18m 28s) (14600 73%) 0.2973 0.54
50m 15s (- 18m 7s) (14700 73%) 0.4950 0.58
50m 32s (- 17m 45s) (14800 74%) 0.2438 0.20
50m 52s (- 17m 24s) (14900 74%) 0.1456 0.55
51m 11s (- 17m 3s) (15000 75%) 0.2142 0.12
51m 34s (- 16m 44s) (15100 75%) 0.0065 0.01
51m 58s (- 16m 24s) (15200 76%) 0.1788 0.73
52m 20s (- 16m 4s) (15300 76%) 0.5392 0.51
52m 43s (- 15m 44s) (15400 77%) 0.1456 0.12
53m 8s (- 15m 25s) (15500 77%) 0.8397 0.92
53m 29s (- 15m 5s) (15600 78%) 0.2879 0.52
53m 48s (- 14m 44s) (15700 78%) 0.1541 0.40
54m 9s (- 14m 23s) (15800 79%) 0.0166 0.13
54m 33s (- 14m 4s) (15900 79%) 0.0741 0.09
54m 56s (- 13m 44s) (16000 80%) 0.2516 0.36
55m 17s (- 13m 23s) (16100 80%) 0.1185 0.10
55m 36s (- 13m 2s) (16200 81%) 0.3239 0.26
56m 0s (- 12m 42s) (16300 81%) 0.4213 0.39
56m 22s (- 12m 22s) (16400 82%) 0.5585 0.27
56m 42s (- 12m 1s) (16500 82%) 0.1293 0.25
57m 3s (- 11m 41s) (16600 83%) 0.0072 0.03
57m 22s (- 11m 20s) (16700 83%) 0.1488 0.11

In [60]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_100_392.pth')

In [61]:
# ??? (h = 100, s = 100) / 10745  (h = 75, s = 50)
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10745, 15, 8, 0, 35, 95, 53, 581, 100, 189, 381, 794, 530, 288, 1350, 842, 787, 606, 859, 305, 330, 279, 1, 9, 182, 211, 41, 56, 1, 48, 39, 0, 0, 0, 3, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25]


### 6.3.6 Remaining errors

In [47]:
questions_train = list(set([' '.join(pair[0]) for dialogue in dialogues_Master for pair in dialogue]))
len(questions_train)

1364

In [48]:
questions_test = list(set([' '.join(pair[0]) for dialogue in dialogues_Master_test for pair in dialogue]))
len(questions_test)

319

In [51]:
intersect = [el for el in questions_test if el in questions_train]
len(intersect)

6

In [70]:
chatbot.eval()
errors_test = trainer.DialoguesWithErrors(chatbot, dialogues_Master_test_var)
with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\training_errors\\M2DS_2019_final_test_errors.file', 'wb') as fileObject :
    pickle.dump(errors_test, fileObject)

2m 56s (- 26m 28s) (100 10%)
5m 59s (- 23m 59s) (200 20%)
8m 48s (- 20m 33s) (300 30%)
11m 31s (- 17m 17s) (400 40%)
14m 31s (- 14m 31s) (500 50%)
17m 42s (- 11m 48s) (600 60%)
20m 32s (- 8m 48s) (700 70%)
23m 14s (- 5m 48s) (800 80%)
26m 15s (- 2m 55s) (900 90%)
29m 18s (- 0m 0s) (1000 100%)


In [84]:
questions = list(set([' '.join(dialogues_Master_test[k][l][0]) for k in list(errors_test.keys()) for l in errors_test[k]]))
len(questions)

183

In [98]:
questions[23]

'faut il valider toutes les matieres pour valider le master ?'

In [125]:
localisation = {}
for question in questions : localisation[question] = []
for i, dialogue in enumerate(dialogues_Master) :
    for j in range(len(dialogue)) :
        q = ' '.join(dialogues_Master[i][j][0])
        if q in questions : localisation[q].append([i, j])

In [146]:
localisation["faut il valider toutes les matieres pour valider le master ?"]

[]

In [144]:
for key in localisation :
    N = len(localisation[key])
    if N > 0 : print(key, " | nombre d'occurences trouvées : {}".format(N))

je suis de lyon 1  | nombre d'occurences trouvées : 4377
j ' aimerais faire le parcours informatique  | nombre d'occurences trouvées : 8988
je souhaite faire le parcours informatique  | nombre d'occurences trouvées : 7610
je souhaite faire le parcours mathematique  | nombre d'occurences trouvées : 7696
non  | nombre d'occurences trouvées : 13554


In [72]:
extended_errors = {}
for i, dialogue in enumerate(dialogues_Master) :
    sim_errors = [j for j, pair in enumerate(dialogue) if ' '.join(pair[0]) in questions]
    if sim_errors != [] : extended_errors[i] = sim_errors

In [73]:
len(extended_errors)

34768

In [74]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 10000, learning_rate=0.025, dic = extended_errors)
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 5000, learning_rate=0.025)

0m 18s (- 31m 16s) (100 1%) 0.0041 0.01
0m 36s (- 30m 9s) (200 2%) 0.0001 0.00
0m 54s (- 29m 14s) (300 3%) 0.0001 0.00
1m 11s (- 28m 45s) (400 4%) 0.0002 0.00
1m 30s (- 28m 32s) (500 5%) 0.0001 0.00
1m 47s (- 27m 57s) (600 6%) 0.0001 0.00
2m 5s (- 27m 53s) (700 7%) 0.0001 0.00
2m 23s (- 27m 35s) (800 8%) 0.0001 0.00
2m 42s (- 27m 26s) (900 9%) 0.0001 0.00
2m 59s (- 26m 59s) (1000 10%) 0.0002 0.00
3m 19s (- 26m 52s) (1100 11%) 0.0002 0.00
3m 37s (- 26m 36s) (1200 12%) 0.0002 0.00
3m 57s (- 26m 27s) (1300 13%) 0.0001 0.00
4m 17s (- 26m 21s) (1400 14%) 0.0001 0.00
4m 35s (- 26m 2s) (1500 15%) 0.0002 0.00
4m 54s (- 25m 46s) (1600 16%) 0.0002 0.00
5m 13s (- 25m 29s) (1700 17%) 0.0001 0.00
5m 31s (- 25m 10s) (1800 18%) 0.0001 0.00
5m 49s (- 24m 52s) (1900 19%) 0.0002 0.00
6m 6s (- 24m 27s) (2000 20%) 0.0012 0.02
6m 25s (- 24m 10s) (2100 21%) 0.0003 0.00
6m 44s (- 23m 54s) (2200 22%) 0.0001 0.00
7m 4s (- 23m 41s) (2300 23%) 0.0001 0.00
7m 23s (- 23m 25s) (2400 24%) 0.0062 0.02
7m 43s (- 23m 1

In [132]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_chatbot_150_per_dialogue.pth')

## 6.4 test

### 6.4.1 Error counts



In [51]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [10094, 27, 13, 0, 12, 20, 31, 99, 86, 190, 197, 600, 686, 622, 1292, 1065, 508, 1046, 644, 435, 322, 392, 0, 23, 188, 514, 46, 66, 20, 41, 0, 0, 0, 0, 270, 189, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 44]


In [75]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [9930, 20, 135, 0, 0, 42, 124, 77, 117, 219, 430, 675, 933, 398, 1878, 605, 897, 972, 677, 499, 149, 293, 2, 9, 215, 258, 2, 56, 0, 34, 0, 0, 0, 0, 12, 82, 71, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 42]


In [52]:
with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\training_errors\\M2DS_2019_final_test_errors.file', 'wb') as fileObject :
    pickle.dump(z, fileObject)

### 6.4.2 Visualization of attention

In [34]:
def showAnswer(model, dialogue):
    answer, log_probas, attn1_w, attn2_w, _ = model.answerTrain(dialogue)
    dialogue = model.flattenDialogue(dialogue)
    print('User : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-2][:-1]]))
    print('target : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-1][:-1]]))
    print('predic : ', ' '.join([model.lang.index2word[int(word)] for word in answer]))
    return

In [ ]:
# n_errors = 1 : info <-> math
# n_errors = 2 : pas grave
# n_errors = 4 : h par semaises / en tout
# n_errors = 5 : combien de projets
# n_errors = 7 : taux de réussite <-> métier de data scientist / ceremonie / remuneration de stage / rapports
# n_errors = 8 : pas grave
# n_errors = 9 : e-candidat / scolarité
# n_errors = 11 : 'oui , j ' ai etudie a unk_token'
# n_errors = 12 : UNK !


In [44]:
chatbot.eval()
n_errors = 14
i = 724
dialogue = dialogues_Master_test_var[z[n_errors][i][0]][:z[n_errors][i][1]+1]
#chatbot.showAttention(dialogue)
showAnswer(chatbot, dialogue)

User :  y a t - il des conseils a avoir sur le master ?
target :  souhaitez - vous suivre le parcours informatique ou mathematiques ?
predic :  le master n ' est pas tourne vers l ' intelligence artificielle .


In [37]:
chatbot.eval()
i = 10
dialogue = dialogues_Master_test_var[i][:-3]
chatbot.showAttention(dialogue)

ValueError: too many values to unpack (expected 4)

### 6.4.3 Interactive mode

write **quit** or **q** or **eoc** (end of conversation) to close conversation interface :

In [86]:
def ReplaceMotVar(motsVar, raw_sentence):
    sentence = []
    word_list = raw_sentence.split(' ')
    for word in word_list :
        if word in motsVar.keys() :
            sentence.append(motsVar[word])
        else :
            sentence.append(word)
    return ' '.join(sentence)


def repair(sentence) :
    s = re.sub(" ' ", "'", sentence)
    s = re.sub(" - ", "-", s)
    s = re.sub(" ,", ",", s)
    s = re.sub(r'(?<=\d) \. (?=\d)', '.', s)
    s = re.sub(" \.", ".", s)
    s = re.sub("\( ", "(", s)
    s = re.sub(" \)", ")", s)
    s = s[0].upper() + s[1:]
    return s


def InteractiveEvaluation(agent) :
    agent.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [128]:
chatbot.eval()
InteractiveEvaluation(chatbot)

comment on peut payer ?
Bot  : Vous pouvez payer par carte, par cheque bancaire, par virement ou en espece.
quels sont les modes de payement
Bot  : Le stage doit avoir une duree de 4 a 6 mois.
combien y a - t - il d ' ects par cours ?
Bot  : Il y a en tout 400h de cours.
combien d'etcs
Bot  : Il y a en tout 400h de cours.
le nombre d'etcs
Bot  : D'accord, comment puis-je vous aider ?
le nombre d etcs est de combien ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
etcs
Bot  : Avez-vous d'autres questions ?
oui, est ce que c'est difficile ?
Bot  : Non, mais il faut travailler assidument.
ca veur dire qu'il faut travailler jours et nuits ?
Bot  : Il faut que vous deposiez votre candidature directement aupres de campus france.
quit


# 7 Noise filtering

## 7.1 Noisy data generation

In [129]:
random.seed(42)

In [130]:
def randomMask(clean_sentences, rand = 0.2) :
    def choseWord(word) :
        return word if random.random() >= rand else 'UNK'
    masked_sentences = []
    for sentence in clean_sentences :
        words = [choseWord(word) for word in sentence.split()]
        masked_sentences.append(' '.join(words))
    return masked_sentences



def randomNoise(clean_sentences, lang, prop = 1.0):
    noisy_sentences = []
    for sentence in clean_sentences :
        if len(sentence.split()) >= 5 :
            words = [lang.index2word[random.randint(3, lang.n_words-1)] for _ in sentence.split()]
            words = [word for i, word in enumerate(words) if i < 5 or random.random() < prop]
            rand_unk = random.randint(1, len(words)-1)
            words.insert(rand_unk, 'UNK')
            if rand_unk > len(words)/ 2 : words += random.choice(['?', '.'])
            noisy_sentences.append(' '.join(words))
    return noisy_sentences

In [131]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

masked_sentences_000 = []
masked_sentences_015 = []
masked_sentences_025 = []
noisy_sentences_100 = []
noisy_sentences_050 = []

for _ in range(10) :
    masked_sentences_000 += clean_sentences
    masked_sentences_015 += randomMask(clean_sentences, rand = 0.15)
    masked_sentences_025 += randomMask(clean_sentences, rand = 0.25)

    noisy_sentences_100 += randomNoise(clean_sentences, lang_M2DS, prop = 1.0)
    noisy_sentences_050 += randomNoise(clean_sentences, lang_M2DS, prop = 0.5)

1364


In [134]:
discriminator_dataset = [[sentence, 1] for sentence in masked_sentences_000 + masked_sentences_015 + masked_sentences_025] + \
                        [[sentence, 0] for sentence in noisy_sentences_100 + noisy_sentences_050]
random.shuffle(discriminator_dataset)
len(discriminator_dataset)

66900

In [135]:
discriminator_dataset[7][0]

"qui enseigne UNK ' ue big data UNK UNK"

## 7.2 Noise filtering model

In [136]:
from chatNLP.misc.Noise_Filter import NoiseFilter, NoiseFilterTrainer, NoiseFilterWrapper

In [138]:
noise_filter = NoiseFilter(chatbot, pretrained = True, layers = [50, 50]).to(device)

In [139]:
noise_filter.nbParametres()

12702

In [ ]:
#noise_filter.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth'))

In [140]:
noise_filter_trainer = NoiseFilterTrainer(device,
                                          criterion = nn.NLLLoss(), 
                                          optimizer = optim.SGD)

In [141]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 20000, learning_rate=0.01)

NameError: name 'time' is not defined

In [ ]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.005)
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.0025)

In [ ]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.001)

In [ ]:
torch.save(noise_filter.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth')

## 7.3 Noise filtering integration

model built on top of a pretrained chatbot model

In [ ]:
def SmartInteractiveEvaluation(agent) :
    agent.chatbot.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [ ]:
noise_filter.eval()
smart_chatbot = NoiseFilterWrapper(noise_filter, chatbot)
smart_chatbot.eval()
SmartInteractiveEvaluation(smart_chatbot)